In [ ]:
# Step 1: Install and Import Dependencies
!pip install sentence-transformers datasets gradio nltk numpy torch networkx scikit-learn pandas



In [ ]:
import torch
import numpy as np
import nltk
import networkx as nx
import gradio as gr
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
from nltk.tokenize import word_tokenize
from datasets import load_dataset

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

#### Load and Preprocess Dataset

In [ ]:
dataset = load_dataset("yahoo_answers_qa")
tech_categories = ["Computers & Internet", "Consumer Electronics", "Yahoo! Products"]
tech_dataset = dataset['train'].filter(lambda x: x['main_category'] in tech_categories)
tech_dataset = tech_dataset.select(range(5000))  # Limit to 5,000 entries
print(f"Loaded {len(tech_dataset)} tech-related entries")

questions = [q['question'] for q in tech_dataset]
answers = [q['answer'] for q in tech_dataset]

#### Create Embeddings Model

In [ ]:
print("Initializing model...")
model = SentenceTransformer("all-MiniLM-L6-v2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("Computing embeddings...")
doc_embeddings = model.encode(answers, convert_to_tensor=True)
print("Embeddings computed successfully!")



### Implement Context Retention

In [ ]:
context_memory = []

def conversational_search(user_query):
    global context_memory
    context_memory.append(user_query)
    if len(context_memory) > 3:
        context_memory.pop(0)
    return search_with_context(user_query, context_memory)

#### Enhance Contextual Understanding

In [ ]:
def search_with_context(user_query, context_memory):
    query_embedding = model.encode(user_query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, doc_embeddings)[0]
    best_match_idx = torch.argmax(similarities).item()

    if len(context_memory) > 1:
        prev_query = context_memory[-2]
        prev_query_embedding = model.encode(prev_query, convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, prev_query_embedding)[0].item()
        if similarity > 0.5 and ("it" in user_query.lower() or len(user_query.split()) < 4):
            prev_tokens = word_tokenize(prev_query.lower())
            subject = next((t for t in prev_tokens if t in knowledge_graph), "issue")
            return f"It seems you're referring to '{subject}' from before: {answers[best_match_idx]}"
    return answers[best_match_idx]

### Implement Advanced Intent Recognition

In [ ]:
intent_mapping = {
    "repair": ["fix", "repair", "broken", "not working", "cracked", "black"],
    "install": ["install", "setup", "put", "add"],
    "diagnose": ["why", "what", "how come", "problem", "issue", "trouble"],
    "replace": ["replace", "change", "new"],
    "update": ["update", "upgrade", "latest"]
}

def identify_intent(user_query):
    tokens = word_tokenize(user_query.lower())
    for intent, keywords in intent_mapping.items():
        if any(word in tokens for word in keywords):
            return intent
    return "general"

#### Build Knowledge Graph

In [ ]:

knowledge_graph = {
    "phone": {"related": ["screen", "battery", "software"], "actions": ["fix", "replace"]},
    "screen": {"related": ["phone", "laptop"], "issues": ["black", "cracked"], "actions": ["fix", "replace"]},
    "laptop": {"related": ["screen", "battery", "windows"], "actions": ["fix", "update"]},
    "windows": {"related": ["laptop", "computer"], "actions": ["install", "update"]},
    "battery": {"related": ["phone", "laptop"], "issues": ["dead"], "actions": ["replace"]},
    "software": {"related": ["phone", "windows"], "issues": ["crash"], "actions": ["update"]},
    "fix": {"targets": ["phone", "screen", "laptop"]},
    "replace": {"targets": ["screen", "battery"]},
    "update": {"targets": ["windows", "software"]}
}

def enhance_with_knowledge(user_query):
    words = word_tokenize(user_query.lower())
    related_terms = []
    for word in words:
        if word in knowledge_graph:
            related_terms.extend(knowledge_graph[word].get("related", []))
    return f"Related concepts: {', '.join(set(related_terms))}" if related_terms else ""

#### Implement Recommendation System

In [ ]:
def recommend_related_queries(user_query):
    try:
        tokens = word_tokenize(user_query.lower())
        intent = identify_intent(user_query)
        recommendations = []

        for token in tokens:
            if token in knowledge_graph:
                related = knowledge_graph[token].get("related", [])
                actions = knowledge_graph[token].get("actions", [])
                for rel in related:
                    if intent in ["repair", "fix"]:
                        recommendations.append(f"How to fix a {rel}?")
                    elif intent == "replace":
                        recommendations.append(f"How to replace a {rel}?")
                    elif intent == "install":
                        recommendations.append(f"How to install {rel}?")
                    elif intent == "update":
                        recommendations.append(f"How to update {rel}?")
                for act in actions:
                    recommendations.append(f"How to {act} a {token}?")

        return list(dict.fromkeys(recommendations))[:3]
    except Exception as e:
        print(f"Error in recommend_related_queries: {e}")
        return []

#### Chatbot Response and UI

In [ ]:
def chatbot_response(user_query, chat_history):
    if not user_query.strip():
        return chat_history + "<div style='margin: 10px; color: red;'>Please enter a query.</div>"

    response = conversational_search(user_query)
    intent = identify_intent(user_query)
    knowledge_info = enhance_with_knowledge(user_query)
    recommendations = recommend_related_queries(user_query)

    message = f"""
    <div style='margin: 10px; padding: 10px; border-radius: 5px;'>
        <strong>Intent:</strong> <span style='color: #1e90ff;'>{intent}</span><br>
        <strong>Response:</strong> {response}<br>
        <strong>Related:</strong> {knowledge_info.replace('Related concepts: ', '') if knowledge_info else 'None'}<br>
        <strong>Recommendations:</strong><br>
    """
    if recommendations:
        for rec in recommendations:
            message += f"""
            <button onclick="document.getElementById('input-textbox').value='{rec}'; document.getElementById('submit-btn').click();"
            style='margin: 5px; padding: 5px; background-color: #add8e6; color: black; border: none; border-radius: 3px; cursor: pointer;'>{rec}</button>
            """
    else:
        message += "None"
    message += "</div>"

    chat_history += f"<div style='margin: 10px; '><strong>You:</strong> {user_query}</div>"
    chat_history += f"<div style='margin: 10px;'><strong>Bot:</strong> {message}</div>"
    return chat_history

with gr.Blocks() as demo:
    gr.Markdown("### Tech Support Chatbot")
    chat_output = gr.HTML(value="", label="Conversation")
    with gr.Row():
        input_box = gr.Textbox(placeholder="e.g., How do I fix my phone?", label="Type here", elem_id="input-textbox")
        submit_btn = gr.Button("Send", elem_id="submit-btn")

    chat_state = gr.State(value="")

    def submit_query(user_query, chat_history):
        if not user_query.strip():
            return "", chat_history
        updated_history = chatbot_response(user_query, chat_history)
        return "", updated_history

    submit_btn.click(fn=submit_query, inputs=[input_box, chat_state], outputs=[input_box, chat_state])
    input_box.submit(fn=submit_query, inputs=[input_box, chat_state], outputs=[input_box, chat_state])
    chat_state.change(fn=lambda x: x, inputs=chat_state, outputs=chat_output)

demo.launch()